## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Открытие файла с данными и изучение общей информации. 

In [1]:
import pandas as pd
bank = pd.read_csv('/datasets/data.csv')
bank.info()
bank.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


на данном этапе я прочитала файл с данными, изучила структуру столбцы, строки
количество значений для двух столбцов days_employed и total_income меньше, чем в остальных столбцах, соответственно можно сделать вывод, что там присутствуют пропуски

### Шаг 2. Предобработка данных

### Обработка пропусков

In [281]:
bank['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [282]:
bank['days_employed'].unique()

array([-8437.67302776, -4024.80375385, -5623.42261023, ...,
       -2113.3468877 , -3112.4817052 , -1984.50758853])

In [283]:
bank['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [284]:
bank['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [285]:
bank['education_id'].unique()

array([0, 1, 2, 3, 4])

In [286]:
bank['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [287]:
bank['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [288]:
bank['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [289]:
bank['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [290]:
bank['debt'].unique()

array([0, 1])

In [291]:
bank['total_income'].unique()

array([253875.6394526 , 112080.01410244, 145885.95229686, ...,
        89672.56115303, 244093.05050043,  82047.41889948])

In [292]:
bank['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [293]:
bank.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

видим, что пропуски присутствуют в столбцах days_employed и total_income, при этом они одинаковы. это говорит о том, что возможно не все клиенты банка работают официально и так же официально получают зарплату, либо вообще не работают( декрет, студенты итд)

In [294]:
bank['days_employed'].value_counts()

-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64

In [295]:
for row in bank:
    print(bank[row].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64
среднее   

здесь я считаю уникальные значения для всех строк и нахожу, что нужно поправить.
children - не верное отрицательное значение -1;
days_employed - присутсвуют пропущенные значения;
total_income - тоже пропущенные значения;
education - разный регистр у букв в словах;
purpose - есть много дублирующих друг друга значений, которые нужно объединять в одни слова для дальнейшей сортировки без потерь.


здесь я нашла средние значения для столбцов days_employed и total_income и заменила этими средними значениями пропущенные значения в этих столбцах

## Замена типа данных

In [302]:
bank['days_employed'] = pd.to_numeric(bank['days_employed'], errors = 'coerce')
bank['total_income'] = pd.to_numeric(bank['total_income'], errors = 'coerce')

данные заменялись на целочисленные методом .to_numeric() с применением значения ошибки как coerce, что позволит  принудительно заменить все некорректные значения на NaN

In [303]:
days_employed_mean = bank['days_employed'].mean()

In [304]:
bank['days_employed'] = bank['days_employed'].fillna(value = days_employed_mean)
print(bank['days_employed'])

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21520     -4529.316663
21521    343937.404131
21522     -2113.346888
21523     -3112.481705
21524     -1984.507589
Name: days_employed, Length: 21525, dtype: float64


In [305]:
total_income_mean = bank['total_income'].mean()
print(total_income_mean)

167422.30220817294


In [306]:
bank.loc[bank['total_income']  == 'NaN', 'total_income'] = total_income_mean
print(bank['total_income'])

0        253875.639453
1        112080.014102
2        145885.952297
3        267628.550329
4        158616.077870
             ...      
21520    224791.862382
21521    155999.806512
21522     89672.561153
21523    244093.050500
21524     82047.418899
Name: total_income, Length: 21525, dtype: float64


### Вывод

здесь я нашла средние значения для столбцов days_employed и total_income и заменила этими средними значениями пропущенные значения в этих столбцах

## Обработка дубликатов

In [307]:
bank.duplicated().sum()
print('Дубликатов в таблице:', bank.duplicated().sum())

Дубликатов в таблице: 54


вычисляем количество дубликатов

In [308]:
print(bank['children'].value_counts())
print(bank['family_status'].value_counts())


 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [309]:
bank['education'] = bank['education'].str.lower()

In [310]:
bank['family_status'] = bank['family_status'].str.lower()

In [311]:
bank['income_type'] = bank['income_type'].str.lower()

In [312]:
bank['purpose'] = bank['purpose'].str.lower()

нужно привезти к нижнему регистру все столбцы, где присуьствуют данные, которые могут быть записаны по разному и соответственно не учитываться в выборке


In [313]:
print(bank['education'].value_counts())
print(bank['family_status'].value_counts())
print(bank['income_type'].value_counts())
print(bank['purpose'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64
женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   653
операции с коммерческой недвижимостью     651
покупка жилья                       

нашли все дубликаты в столбцах после приведения к нижнему регистру 
выяснили, что в столбцах education, family_status и income_type нет лишних дубликатов, 
а в столбце purpose множество дублирующих друг друга вариаций названий целей для выдачи кредита

### Вывод

данные показали, что удалять дубли нужно там, где нарушен регистр при написании значений. 
в столбце purpose как раз найдено множество значений, которые дублируют одни и те же цели кредита, следовательно следующим гагом в работе должна быть лемматизация данных столбца purpose

### Лемматизация

In [314]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()

def lemm_purpose(row):
    lemmas = ' '.join(m.lemmatize(row))
    return lemmas

bank['lemm_purpose'] = bank['purpose'].apply(lemm_purpose)
print(bank['lemm_purpose'])
print(Counter(' '.join(bank['lemm_purpose'].unique()).split()))


0                         покупка   жилье \n
1               приобретение   автомобиль \n
2                         покупка   жилье \n
3            дополнительный   образование \n
4                       сыграть   свадьба \n
                        ...                 
21520                операция   с   жилье \n
21521             сделка   с   автомобиль \n
21522                        недвижимость \n
21523    на   покупка   свой   автомобиль \n
21524           на   покупка   автомобиль \n
Name: lemm_purpose, Length: 21525, dtype: object
Counter({'покупка': 10, 'недвижимость': 10, 'образование': 9, 'автомобиль': 8, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


In [315]:
def categor_purpose(row):
    purpose = row['lemm_purpose']
    if 'ремонт' in purpose:
        return 'ремонт'
    elif 'авто' in purpose or 'машин' in purpose:
        return 'автомобиль'
    elif 'квартир' in purpose or 'дом' in purpose or 'недвиж' in purpose or 'жиль' in purpose:
        return 'недвижимость'
    elif 'образ' in purpose:
        return 'образование'
    elif 'свадьб' in purpose:
        return 'свадьба'
    else:
        return purpose
    
bank['lemm_purpose'] = bank.apply(categor_purpose, axis=1)   
print(bank['lemm_purpose'].value_counts())



#bank['lemm_purpose'] = bank.apply(categor_purpose, axis=1)
#print(bank['lemm_purpose'].value_counts())

недвижимость    10228
автомобиль       4315
образование      4022
свадьба          2348
ремонт            612
Name: lemm_purpose, dtype: int64


для проведения лемматизиции данных всего столбца purpose нужно создать цикл, который проходится по всей длине столбца.
добавляем новый столбец с лемматизированными названиями к нашей таблице и выводим его на экран


в результате получили список запросов на получение кредита: недвижимость, свадьба, автомобиль, образование, ремонт

### Категоризация данных

In [316]:
#создадим словари для разных столбцов в соответствии с поставленными вопросами
#сначала рассмотрим детей и разделим их на категории: 
print(bank['children'].value_counts())
#children_grouped = bank.groupby('children').count()
#print(children_grouped)
def children_status(children):
    if children <= 0:
        return 'бездетная'
    if children <= 2:
        return 'малодетная'
    if children >= 3:
        return 'многодетная'
        return 'есть'
 
bank['children_status'] = bank['children'].apply(children_status)
#children_grouped['children_status'] = children_grouped['children'].apply(children_status)
print(bank['children_status'].head(20))
print(children_status(5)) #проверяю, что сортировка работает и выдает нужную категорию при запросе
print(children_status(0))

#теперь распределим по категориям в браке\не в браке:
def family_status(value):
    if value == 'женат / замужем':
        return 'в браке'
    return 'не в браке'
      
bank['family_status_new'] = bank['family_status'].apply(family_status)
print(bank['family_status_new'])
print(family_status('женат / замужем')) #проверяем, как работает функция

#далее категория с суммарным доходом(до этого выяснила,что среднее число дохода 167422,на этом и основываю свои распределения):
def income_define(sum):
    if sum <= 100000:
        return 'низкий доход'
    if sum <= 170000:
        return 'средний доход'
    return 'высокий доход'

bank['total_income_new'] = bank['total_income'].apply(income_define)
print(bank['total_income_new'])
print(income_define(200000)) #проверяю работу функции

#то же самое делаем для столбца задолженностьчтобы потом вывести его в сводной таблице
def debt_count(status):
    if status == 0:
        return 'в срок'
    return 'не в срок'

bank['debt_status'] = bank['debt'].apply(debt_count)
print(bank['debt_status']) #проверка



 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
0      малодетная
1      малодетная
2       бездетная
3     многодетная
4       бездетная
5       бездетная
6       бездетная
7       бездетная
8      малодетная
9       бездетная
10     малодетная
11      бездетная
12      бездетная
13      бездетная
14      бездетная
15     малодетная
16      бездетная
17      бездетная
18      бездетная
19      бездетная
Name: children_status, dtype: object
многодетная
бездетная
0           в браке
1           в браке
2           в браке
3           в браке
4        не в браке
            ...    
21520    не в браке
21521       в браке
21522    не в браке
21523       в браке
21524       в браке
Name: family_status_new, Length: 21525, dtype: object
в браке
0        высокий доход
1        средний доход
2        средний доход
3        высокий доход
4        средний доход
             ...      
21520    высокий доход
21521

Данные сгруппированы по категориям, теперь, при ответе на вопросы ниже можо использовать данную группировку.
Группировку произвела в соответствиии с величинами

### Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [317]:
data_pivot_children = bank.pivot_table(index=['children_status'], 
                                       columns='debt_status', 
                                       values='debt', 
                                       aggfunc='count')
#создаю сводную таблицу для соотнесения данных возврата в срок\не в срок и наличия и количества детей
data_pivot_children['%'] = (data_pivot_children['в срок'] / (data_pivot_children['не в срок'] + data_pivot_children['в срок'])) * 100
print(data_pivot_children) #вычисляем процент возврата кредита в срок по разным категориям

debt_status      в срок  не в срок          %
children_status                              
бездетная         13132       1064  92.504931
малодетная         6235        638  90.717300
многодетная         417         39  91.447368


Из результатов видно, что кредит чаще возвращают люди без детей, что объясняется отсутствием статьи расходов на ребенкка и вследствии больше свободных денег;

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [318]:
data_pivot_family = bank.pivot_table(index=['family_status_new'], 
                                       columns='debt_status', 
                                       values='debt', 
                                       aggfunc='count')
#создаю сводную таблицу для соотнесения данных возврата в срок\не в срок и сеймейного статуса
data_pivot_family['%'] = (data_pivot_family['в срок'] / (data_pivot_family['не в срок'] + data_pivot_family['в срок'])) * 100
print(data_pivot_family)  #вычисляем процент возврата кредита в срок по разным категориям

debt_status        в срок  не в срок          %
family_status_new                              
в браке             11449        931  92.479806
не в браке           8335        810  91.142701


Из результатов видно, что люди, находящиеся в браке чаще погашают кредиты без долгов, что можно объяснить тем, что у супругов часто общий бюджет и соответственно о возврате кредита обеспокоены оба;

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [319]:
data_pivot_income = bank.pivot_table(index=['total_income_new'], 
                                       columns='debt_status', 
                                       values='debt', 
                                       aggfunc='count')
#создаю сводную таблицу для соотнесения данных возврата в срок\не в срок и общего дохода по категориям
data_pivot_income['%'] = (data_pivot_income['в срок'] / (data_pivot_income['не в срок'] + data_pivot_income['в срок'])) * 100
print(data_pivot_income) #вычисляем процент возврата кредита в срок по разным категориям

debt_status       в срок  не в срок          %
total_income_new                              
высокий доход       8673        717  92.364217
низкий доход        4109        354  92.068116
средний доход       7002        670  91.266945


Из результатов можно сделать вывод, что люди с высоким и низким доходом расплачивются с кредитом без задолженностей. Это можно объяснить тем, что люи с высоким доходом в принципе располагают достаточными финансами для возврата, а люди с низким доходом имеют на деле доход выше, чем предоставлено ьанку в документах, потому что часто у людей может быть не серая зарплата либо не официальный дополнительный доход( заработок в интерснете, в соц сетях, сдача квартиры в аренду итд)

- Как разные цели кредита влияют на его возврат в срок?

In [320]:
data_pivot_purpose = bank.pivot_table(index=['lemm_purpose'], 
                                       columns='debt_status', 
                                       values='debt', 
                                       aggfunc='count')
#создаю сводную таблицу для соотнесения данных возврата в срок\не в срок и общего дохода по категориям
data_pivot_purpose['%'] = (data_pivot_purpose['в срок'] / (data_pivot_purpose['не в срок'] + data_pivot_purpose['в срок'])) * 100
print(data_pivot_purpose) #вычисляем процент возврата кредита в срок по разным категориям

debt_status   в срок  не в срок          %
lemm_purpose                              
автомобиль      3912        403  90.660487
недвижимость    9481        747  92.696519
образование     3652        370  90.800597
ремонт           577         35  94.281046
свадьба         2162        186  92.078365


Из результата можно сделать вывод, что лучше всего отдают кредиты за ремонт, затем идут свадьба и недвижимось. Хуже всего возвращают кредиты за автомобиль и образование. Это можно объяснить тем, что ремонт после того, как он сделан либо квартира куплена, больше не требуют доп расходов и креит можно спокойно платить, в то время как машина требует регулярных денежных вложений в процессе ее эксплуатации, а так же человек пока учится не имеет полноценной возможности работать и соответственно не может возвращать все в срок

### Шаг 4. Общий вывод

Общий вывод можно сделать такой: банкам лучше всего выдавать кредиты на недвижимость, ремонт и свадьбу - эти виды кредитов выплачивают лучше всего.